<a href="https://colab.research.google.com/github/nedokormysh/Stepik_Ai_edu_RNN/blob/week_4_PyTorch/PyTorch_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задача классификации (MNIST)

В этом ноутбуки вы обучите полносвязную нейронную сеть для решения задачи классификации на датасете MNIST.

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [ ]:
# Check Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define Hyper-parameters
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [ ]:
# MNIST dataset

train_dataset = torchvision.datasets.MNIST(root='../../data',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data',
                                          train=False,
                                          transform=transforms.ToTensor())

100%|██████████| 9912422/9912422 [00:00<00:00, 87715801.96it/s]


Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 21728375.57it/s]


Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 49314684.80it/s]


Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 16994227.27it/s]

Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw



In [ ]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [ ]:
# Fully connected neural network
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backprpagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.3349
Epoch [1/5], Step [200/600], Loss: 0.2947
Epoch [1/5], Step [300/600], Loss: 0.2063
Epoch [1/5], Step [400/600], Loss: 0.2169
Epoch [1/5], Step [500/600], Loss: 0.2238
Epoch [1/5], Step [600/600], Loss: 0.2428
Epoch [2/5], Step [100/600], Loss: 0.1922
Epoch [2/5], Step [200/600], Loss: 0.0481
Epoch [2/5], Step [300/600], Loss: 0.0713
Epoch [2/5], Step [400/600], Loss: 0.0388
Epoch [2/5], Step [500/600], Loss: 0.0756
Epoch [2/5], Step [600/600], Loss: 0.0793
Epoch [3/5], Step [100/600], Loss: 0.1089
Epoch [3/5], Step [200/600], Loss: 0.0994
Epoch [3/5], Step [300/600], Loss: 0.1601
Epoch [3/5], Step [400/600], Loss: 0.0646
Epoch [3/5], Step [500/600], Loss: 0.0188
Epoch [3/5], Step [600/600], Loss: 0.1996
Epoch [4/5], Step [100/600], Loss: 0.0674
Epoch [4/5], Step [200/600], Loss: 0.1270
Epoch [4/5], Step [300/600], Loss: 0.0678
Epoch [4/5], Step [400/600], Loss: 0.0129
Epoch [4/5], Step [500/600], Loss: 0.0222
Epoch [4/5], Step [600/600], Loss:

In [ ]:
# Test the model
# In the test phase, don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Accuracy of the network on the test images: 97.96 %
